In [1]:
# foir the 

In [2]:
import pickle
import pandas as pd
import numpy as np
import nrrd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import clear_output

from radiomics import featureextractor, getTestCase
import SimpleITK as sitk
import subprocess

import os
import glob
import shutil

import time
import datetime

print(datetime.datetime.now())

2022-02-22 18:44:23.771295


In [3]:
path_data = '../data/d20220201/'

# INPUTS
fn_patient = '../metadata/Celine_FINAL_RADIOPOP_DATA_with LRE Death_220126.xlsx'
fn_patient2 = '../metadata/List Celine Radipop 20.12.21.xlsx'
path_masks = path_data + '0raw_masks/'
path_meta = path_data + 'nrrds_metadata/'

# OUTPUTS
path_liverMasks = path_data + 'nrrds_maskLiver/'
path_spleenMasks = path_data + 'nrrds_maskSpleen/'

# PARAMETERS
mask_liverNum = 1
mask_spleenNum = 2

In [4]:
# prepare directories
try:
    os.mkdir(path_liverMasks)
except OSError:
    print ("Creation of the directory %s failed" % path_liverMasks)
else:
    print ("Successfully created the directory %s " % path_liverMasks)
    
try:
    os.mkdir(path_spleenMasks)
except OSError:
    print ("Creation of the directory %s failed" % path_spleenMasks)
else:
    print ("Successfully created the directory %s " % path_spleenMasks)

Creation of the directory data/d20220201/nrrds_maskLiver/ failed
Creation of the directory data/d20220201/nrrds_maskSpleen/ failed


In [5]:
patients_df = pd.read_excel(fn_patient, index_col=0, dtype=str)#, index_col=0, header=1, nrows=371, dtype=str)
patients_df['patientID'] = patients_df.index
patients_df['patientID'] = patients_df['patientID'].apply(lambda x: str(x).zfill(4))
patients_df = patients_df.set_index('patientID')
patients_df

patients_df2 = pd.read_excel(fn_patient2, index_col=0, dtype=str)#, index_col=0, header=1, nrows=371, dtype=str)
patients_df2['temp'] = patients_df2.index
patients_df2['patientID'] = patients_df2['temp'].str.split(" ", n=1).str[0]
patients_df2.patientID.fillna(patients_df2.temp, inplace=True)
patients_df2 = patients_df2[patients_df2['patientID'].astype(int) >= 1000]
patients_df2 = patients_df2.set_index('patientID')
patients_df2

listPatients = patients_df.index.tolist() + patients_df2.index.tolist()
print(len(listPatients))

562


In [6]:
t0 = time.time()
problemsMeta = []
problemsMask = []
maskMissing = []

#listMasks = [x for x in os.listdir(path_masks) if os.path.isdir(os.path.join(path_masks, x))]

for eaPatientID in listPatients:
    curr_path = path_masks + str(eaPatientID)
    opfn_liver = path_liverMasks + str(eaPatientID) + '.nrrd'
    opfn_spleen = path_spleenMasks + str(eaPatientID) + '.nrrd'
    if os.path.exists(opfn_liver) and os.path.exists(opfn_spleen):
        print('ID_' + str(eaPatientID) + ': already stacked.')
    else:
        if os.path.isdir(curr_path):
            fns_masks = [x for x in os.listdir(curr_path) if (x.endswith('.p') and not x.startswith("._"))]
            i_masks = np.array([x.replace(".p", "") for x in fns_masks], dtype = int)
            try:
                mask = np.array([pickle.load(open(os.path.join(curr_path, x), "rb")) 
                                 for x in np.array(fns_masks)[np.argsort(i_masks).astype(int)]])
                mask = mask.transpose()
                # make liver masks
                mask_liver = mask.copy()
                mask_liver[np.where(mask_liver != mask_liverNum)] = 0
                mask_liver[np.where(mask_liver == mask_liverNum)] = 1
                # make spleen masks
                mask_spleen = mask.copy()
                mask_spleen[np.where(mask_spleen != mask_spleenNum)] = 0
                mask_spleen[np.where(mask_spleen == mask_spleenNum)] = 1
                try:
                    # get metadata from other nrrd (converted from dicom)
                    temp_nrrd = nrrd.read(path_meta + str(eaPatientID) + '.nrrd')
                    currMeta = temp_nrrd[1]
                    currMeta["type"] = "int"
                    # write nrrds for spleen and liver masks
                    nrrd.write(path_liverMasks + str(eaPatientID) + '.nrrd', mask_liver, header=currMeta)
                    nrrd.write(path_spleenMasks + str(eaPatientID) + '.nrrd', mask_spleen, header=currMeta)
                    print('ID_' + str(eaPatientID) + ': success! masks stacked.')
                except:
                    print('ID_' + str(eaPatientID) + ': metadata file missing')
                    problemsMeta.append(eaPatientID)
            except:
                print('ID_' + str(eaPatientID) + ': problem loading mask')
                problemsMask.append(eaPatientID)
        else:
            print('ID_' + str(eaPatientID) + ': mask files missing')
            maskMissing.append(eaPatientID)
    
    
print(time.time() - t0)
print(datetime.datetime.now())

ID_0001: already stacked.
ID_0002: already stacked.
ID_0004: already stacked.
ID_0005: already stacked.
ID_0006: already stacked.
ID_0007: already stacked.
ID_0008: already stacked.
ID_0009: already stacked.
ID_0010: already stacked.
ID_0011: already stacked.
ID_0013: already stacked.
ID_0015: already stacked.
ID_0016: already stacked.
ID_0018: already stacked.
ID_0019: already stacked.
ID_0020: already stacked.
ID_0021: already stacked.
ID_0022: already stacked.
ID_0023: already stacked.
ID_0024: already stacked.
ID_0025: already stacked.
ID_0026: already stacked.
ID_0027: already stacked.
ID_0028: already stacked.
ID_0029: already stacked.
ID_0030: already stacked.
ID_0031: already stacked.
ID_0032: already stacked.
ID_0033: already stacked.
ID_0034: already stacked.
ID_0035: already stacked.
ID_0036: already stacked.
ID_0037: already stacked.
ID_0038: already stacked.
ID_0039: already stacked.
ID_0040: already stacked.
ID_0041: already stacked.
ID_0042: already stacked.
ID_0043: alr

In [7]:
print('ProblemsMeta:')
print(problemsMeta)

print('ProblemsMask:')
print(problemsMask)

print('MissingMask:')
print(maskMissing)

ProblemsMeta:
[]
ProblemsMask:
[]
MissingMask:
[]


In [8]:
print(datetime.datetime.now())

2022-02-22 18:45:00.578251
